In [21]:
#load images and labels
import pandas as pd
import numpy as np

movies = pd.read_csv('data/mov-cleaned.csv')
rated_movies = movies[movies.vote_count != 0]
posters = rated_movies[['poster_path','vote_average']]

posters = posters.dropna(axis=0,how='any')

In [19]:
from keras.preprocessing import image
path = 'data/small/poster'
X = []
y = []
for i in posters.values:
    try:
        img = image.load_img(path + i[0])
        X.append(image.img_to_array(img))
        y.append(i[1])
    except:
        print(i)
X = np.array(X)
y = np.array(y)

['/9mxGgvWMcXC5LYUkKi3AeP3VTgN.jpg' 6.9]
['/HryOo5fCwJ5QUbNqjYAotgoj9c.jpg' 5.8]


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(3,92,114),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(1, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()


#Then training with your data ! 